This notebook is centered on selecting impact sentences based on a pre-defined threshold specified in 0_input.ipynb. It segregates sentences pertaining to structural and community impacts, preparing them for subsequent summary generation.

Please ensure to paste the input_path, which is the location of the output of the 01_input.ipynb file, at the start of this notebook. This step is the only requirement to load all necessary information for the execution of the code.

Recommended Google Colab Runtime Type: CPU (default).

In [ ]:
# Specify the directory path where the output of the input file 01_input.ipynb was saved
input_path = "/content/drive/My Drive/ImpactDataMining/Hurricane_Otis/01_Input"

All the below sections automatically retrieve data from the 01_input.ipynb file, as well as results from previous notebooks in this series. The code is designed to run using this information, so no further edits are required beyond this point.

In [ ]:
import os
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn

from google.colab import drive
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score, precision_score, recall_score,  f1_score

In [ ]:
import time

start_time = time.time()

In [ ]:
def current_path():
  print("Current working directory")
  print(os.getcwd())
  print()

current_path()
drive.mount('/content/drive')
os.chdir(input_path)
current_path()

Current working directory
/content

Mounted at /content/drive
Current working directory
/content/drive/My Drive/ImpactDataMining/Hurricane_Otis/01_Input



In [ ]:
with open('0_input.json', 'r') as file:
    data = json.load(file)
    output_path = data['output_path']
    labels_struct = data['keywords_struct']
    labels_comm = data['keywords_comm']
    labels_non_impact = data['keywords_non_impact']
    threshold = data['threshold']

In [ ]:
current_path()
drive.mount('/content/drive')
os.chdir(output_path)
current_path()

Current working directory
/content/drive/My Drive/ImpactDataMining/Hurricane_Otis/01_Input

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Current working directory
/content/drive/My Drive/ImpactDataMining/Hurricane_Otis/03_Output



In [ ]:
with open('1_output.json', 'r') as file:
    data = json.load(file)
    text_body = data['text_body']
    text_table = data['text_table']
    idx_body = data['idx_body']
    idx_table = data['idx_table']

with open('2a_output.json', 'r') as file:
    data = json.load(file)
    sent_all = data['sent_all']
    keywords = data['labels']
    label_pred = data['result_probs']
    result_labels = data['result_labels']

In [ ]:
for i, n in enumerate(result_labels):
  if n in labels_non_impact:
    label_pred[i] = 0

In [ ]:
y_pred = [1 if n >= threshold else 0 for n in label_pred]

In [ ]:
idx_flat = [i for i, n in enumerate(y_pred) if n == 1]

idx_para_nested = idx_table + idx_body
idx_len = [len(n) for n in idx_para_nested]
idx_para_flat = [k for n in idx_para_nested for k in n]
idx_count = list(range(len(idx_para_flat)))

In [ ]:
idx_count_nested = []; temp_list = [];
for i, (n, k) in enumerate(zip(idx_para_flat, idx_count)):
  if i > 0 and n <= idx_para_flat[i-1]:
    idx_count_nested.append(temp_list)
    temp_list = []
  temp_list.append(k)

if temp_list:
  idx_count_nested.append(temp_list)

idx_nested = [];
for n in idx_count_nested:
  temp_list = [k for k in n if k in idx_flat]
  if temp_list != []:
    idx_nested.append(temp_list)

In [ ]:
sent_nested = []
for n in idx_nested:
  temp_list = [sent_all[i] for i in n]
  sent_nested.append(temp_list)

In [ ]:
pos_labels = [result_labels[i] for i in idx_flat]

In [ ]:
idx_struct_flat = [n for n, k in zip(idx_flat, pos_labels) if k in labels_struct]

idx_struct_nested = []
for n in idx_nested:
  temp_list = [k for k in n if k in idx_struct_flat]
  if temp_list != []:
    idx_struct_nested.append(temp_list)

sent_struct_nested = []
for n in idx_struct_nested:
  temp_list = [sent_all[i] for i in n]
  sent_struct_nested.append(temp_list)

idx_comm_flat = [n for n, k in zip(idx_flat, pos_labels) if k in labels_comm]

idx_comm_nested = []
for n in idx_nested:
  temp_list = [k for k in n if k in idx_comm_flat]
  if temp_list != []:
    idx_comm_nested.append(temp_list)

sent_comm_nested = []
for n in idx_comm_nested:
  temp_list = [sent_all[i] for i in n]
  sent_comm_nested.append(temp_list)

In [ ]:
print('Structural Impact Sentences')
sent_struct_nested

Structural Impact Sentences


[['Acapulco experienced significant damage, especially to residential and high-rise buildings along the coast.'],
 ['The unprecedented scale of damage to single- and multi-story buildings in low-lying regions of Acapulco resulted in dramatic failures to multiple glazing, cladding, and roof surfaces.'],
 ['These failures extended to nearly every high-rise structure in the nearshore region.'],
 ['overviewing of the hurricane, particularly relating to the wind and storm surge hazards and their impacts on the built environment, framing the regulatory environment and construction practices in the affected area, synthesizing preliminary reports of damage to buildings and other infrastructure, and providing recommendations for continued study of this event by StEER and the wider engineering reconnaissance community.'],
 ['Hurricane Otis left a trail of destruction in its wake, with powerful winds and intense rainfall, leading to widespread flooding and landslides across the region.'],
 ['The 

In [ ]:
print('Community Impact Sentences')
sent_comm_nested

Community Impact Sentences


[['About 80% of the hotels in Acapulco were damaged, and about 274,000 homes were destroyed in the region.'],
 ['Over half a million electricity consumers were affected.'],
 ['Meanwhile, roads, ports and air transit were disrupted, limiting egressing and access to the affected coastal city.'],
 ['The hurricane led to a reported 45 casualties and an estimated $15 billion in damage.'],
 ['One of the hardest-hit areas was the popular beach resort city of Acapulco, located on the Pacific coast of Mexico in the state of Guerrero.'],
 ["According to the governor of Guerrero, the hurricane caused damage to 80% of the hotels in Acapulco, a devastating blow to the city's tourism industry (Mega et al., 2023), with another approximately 274,000 houses destroyed (Reliefweb, 2023).",
  "The hurricane also took a toll on the region's essential services and infrastructure."],
 ["The electric grid was severely affected, leaving 513,544 customers in the State of Guerrero without power in the aftermath 

In [ ]:
# Saving results to a JSON file
with open('2b_output.json', 'w') as file:
    json.dump(
        {'sent_nested': sent_nested, 'sent_struct_nested': sent_struct_nested,
         'sent_comm_nested': sent_comm_nested}, file
        )

In [ ]:
end_time = time.time()
execution_time = end_time - start_time

print("Execution time:", execution_time, "seconds")

Execution time: 22.339475393295288 seconds
